## Initial summary of event files

**Dataset**: Go-nogo categorization and detection task v1.2.0 [openNeuro ds002680](https://openneuro.org/datasets/ds002680/versions/1.2.0).

This script does a preliminary summary of the contents of the events files.
The summary includes printing out the column names of each event file so
that they can be manually checked for differences.

The script assumes that the data is in BIDS format and that each BIDS events
file of the form `_events.tsv` has a corresponding events file with
suffix `_events_temp.tsv` that was previously dumped from the `EEG.set` files.

In order to compare the events coming from the BIDS events files and those
from the EEG.set files, the script creates dictionaries of `key` to full path
for each type of file.  The `key` is of the form `sub-xxx_run-y` which
uniquely specify each event file in the dataset. If a dataset contains
multiple sessions for each subject, the `key` should include additional
parts of the file name to uniquely specify each subject.

Keys are specified by a `entities` tuple lists the BIDS entity names
to include in the key.
BIDS base file names are constructed of entity *name*-*value* pairs separated
by underbars and followed by an ending *_suffix*.

For a file name `sub-001_ses-3_task-target_run-01_events.tsv`,
the tuple ('sub', 'task') gives a key of `sub-001_task-target`,
while the tuple ('sub', 'ses', 'run) gives a key of `sub-001_ses-3_run-01`.
The use of dictionaries of file names with such keys makes it
easier to associate related files in the BIDS naming structure.

The setup requires the setting of the following variables for your dataset:

| Variable | Purpose |
| -------- | ------- |
| bids_root_path | Full path to root directory of dataset.|
| exclude_dirs | List of directories to exclude when constructing file lists. |
| entities  | Tuple of entity names used to construct a unique keys representing filenames. <br>(See [Dictionaries of filenames](https://hed-examples.readthedocs.io/en/latest/HedInPython.html#dictionaries-of-filenames-anchor) for examples of how to choose the keys.)|
| bids_skip_columns  |  List of column names in the `events.tsv` files to skip in the analysis. |
| eeg_skip_columns  | List of column names in the `events_temp.tsv` files form EEG.events to skip in analysis.|

In [1]:
from hed.tools import BidsTabularDictionary, HedLogger, report_diffs
from hed.util import get_file_list

# Variables to set for the specific dataset
bids_root_path = 'G:/GoNogo/GoNogoWorking'
exclude_dirs = ['code', 'stimuli']
entities = ('sub', 'ses', 'run')
bids_skip_columns = ['onset', 'response_time', 'stim_file']
eeg_skip_columns = ['latency', 'response_time', 'stim_file']
log_name = 'go_nogo_01_initial_summary_log'

# Set up the logger
log_file_name = f"code/curation_logs/{log_name}.txt"
logger = HedLogger(name=log_name)

# Construct the event file dictionaries for the BIDS and for EEG.event files
event_files_bids = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_events",
                                 exclude_dirs=exclude_dirs)
bids_dict = BidsTabularDictionary("Go nogo bids", event_files_bids, entities=entities)
event_files_eeg = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_events_temp",
                                exclude_dirs=exclude_dirs)
eeg_dict = BidsTabularDictionary("Go nogo eeg", event_files_eeg, entities=entities)


# Output a list of files for the two cases and detect differences
print(f"\n{report_diffs(bids_dict, eeg_dict, logger)}\n\n")

# Output the column names for each type of event file
print(f"\nBIDS style event file columns:")
for key, file, rowcount, columns in bids_dict.iter_files():
    print(f"{key} [{rowcount} events]: {str(columns)}")

print(f"\nEEG.event style event file columns:")
for key, file, rowcount, columns in eeg_dict.iter_tsv_info():
    print(f"{key} [{rowcount} events]: {str(columns)}")

HedFileError: ('BadKeyValue', 'The piece events is not in key-value form', '')

In [ ]:
from hed.tools import BidsTabularSummary
bids_dicts_all, bids_dicts =  BidsTabularSummary.make_combined_dicts(bids_dict, skip_cols=bids_skip_columns)
print(f"\nSummary of all BIDS events files:\n{bids_dicts_all}")

eeg_dicts_all, eeg_dicts =  BidsTabularSummary.make_combined_dicts(eeg_dict, skip_cols=eeg_skip_columns)
print(f"\nSummary of all BIDS events files:\n{eeg_dicts_all}")